# Connecting structures together

This example shows how to connect neurons in different structures together and is useful when you do not have the long range axons available. It can also be used within a volume, but note that this will then be in addition to what synapses touch detection provides (so useful when no axon is available).

* Define two volumes, ```VolumeA``` and ```VolumeB``` (far apart).
* Fill the volumes with ```dSPN``` and ```iSPN``` neurons respectively.
* Define the connection rule between the two neuron types.
* Create connections

## Define volumes

Here we use ```create_cube_mesh``` to create two cubical volumes with sides 300 micrometer that are 10 mm apart.

In [1]:
import os
from snudda.place import create_cube_mesh

network_path = os.path.join("networks", "connect_structures_network")
mesh_file_a = os.path.join(network_path, "mesh", "volume_A.obj")
mesh_file_b = os.path.join(network_path, "mesh", "volume_B.obj")

create_cube_mesh(mesh_file_a, [5e-3,0,0], 300e-6, "Volume A - connect structures example")
create_cube_mesh(mesh_file_b, [-5e-3,0,0], 300e-6, "Volume B - connect structures example")

## Defining network, including projection

Normally neurons are connected using touch detection, but here we show how to connect neurons in separate volumes together. This can be used to create topographic maps between structures. To define a projection map (here [```data/ExampleProjection.json```](data/ExampleProjection.json)) we define a set of source points, and their corresponding target points. Intermediate points will be interpolated. Note that points outside the convex hull of the source points will have no projection defined.

```
{
    "ExampleProjection" : {
        "source" : [[4850, -150, -150],
                    [4850, -150, 150],
                    [4850, 150, -150],
                    [4850, 150, 150],
                    [5150, -150, -150],
                    [5150, -150, 150],
                    [5150, 150, -150],
                    [5150, 150, 150]],
        
        "destination" : [[-5150, -150, -150],
                         [-5150, -150, 150],
                         [-5150, 150, -150],
                         [-5150, 150, 150],
                         [-4850, -150, -150],
                         [-4850, -150, 150],
                         [-4850, 150, -150],
                         [-4850, 150, 150]] 
    }
    
}
```

Here a neuron at ```[4850, -150, -150]``` will project to a neuron at ```[-5150, -150, -150]```.

We define the structures (```define_structure```) using the mesh files created above, add add the neurons to the two volumes (```add_neurons```).

Normally ```add_neuron_target``` is used to define which neurons we want to couple together using touch detect. For projections we instead use ```neuron_projection```.

* ```neuron_name``` is the presynaptic neuron
* ```target_name``` is the postsynaptic neuron
* ```projection_name``` is the name of the projection, same as in the JSON projection file
* ```projection_file``` defines the JSON projection map file.
* ```source_volume``` volume of presynaptic neuron
* ```dest_volume``` volume of postsynaptic neuron
* ```projection_radius``` any neuron within this distance from target centre can be selected as target
* ```number_of_targets``` is number of target neurons, given as ```[mean, std]```
* ```number_of_synapses``` is number of synapses per connection, give as ```[mean, std]```
* ```dendrite_synapse_density``` is density as a function of ```d``` (distance to soma), evaluated using ```numexpr``` 
* ```connection_type``` is synapse type, e.g. ```GABA```
* ```dist_pruning```, ```f1```, ```soft_max```, ```mu2```, ```a3``` are the standard pruning parameters.

In [2]:
from snudda import SnuddaInit

cnc = SnuddaInit(network_path=network_path, random_seed=123)

cnc.define_structure(struct_name="VolumeA", struct_mesh=mesh_file_a, d_min=15e-6, mesh_bin_width=50e-6)
cnc.define_structure(struct_name="VolumeB", struct_mesh=mesh_file_b, d_min=15e-6, mesh_bin_width=50e-6)

cnc.add_neurons(name="dSPN", num_neurons=20, volume_id="VolumeA",
                neuron_dir=os.path.join("$DATA","neurons","striatum","dspn"))
cnc.add_neurons(name="iSPN", num_neurons=20, volume_id="VolumeB",
                neuron_dir=os.path.join("$DATA","neurons","striatum","ispn"))

# Normally we would use add_neuron_target to connect the neurons in the same volume together, 
# but this time we want to use neuron_projection instead.

proj_file = os.path.join("data", "ExampleProjection.json")

cnc.neuron_projection(neuron_name="dSPN", 
                      target_name="iSPN",
                      projection_name="ExampleProjection",
                      projection_file=proj_file,
                      source_volume="VolumeA",
                      dest_volume="VolumeB",
                      projection_radius=100e-6,
                      number_of_targets=[10, 5],
                      number_of_synapses=[10, 5],
                      dendrite_synapse_density="1",
                      connection_type="GABA",
                      dist_pruning=None,
                      f1=0.9,
                      soft_max=None,
                      mu2=None,
                      a3=None)


cnc.write_json()



Adding neurons: dSPN from dir $DATA/neurons/striatum/dspn
Adding neurons: iSPN from dir $DATA/neurons/striatum/ispn
Writing networks/connect_structures_network/network-config.json


In [3]:
from snudda import SnuddaPlace
sp = SnuddaPlace(network_path=network_path, verbose=False)
sp.parse_config()
sp.write_data()

In [4]:
from snudda.plotting import PlotNetwork
network_file = os.path.join(network_path, "network-neuron-positions.hdf5")
pn = PlotNetwork(network_file)
pn.plot(plot_axon=False, plot_dendrite=False, plot_synapses=False)
pn.close() 

__TODO__: Update logic so that if there are no connections allowed between the neurons present in a hyper voxel, 
then it skips those in detection step.

In [5]:
from snudda import SnuddaDetect

sd = SnuddaDetect(network_path=network_path)
sd.detect()

No d_view specified, running distribute neurons in serial
Processing hyper voxel : 21/592 (20 neurons)
Processing hyper voxel : 549/592 (20 neurons)
Processing hyper voxel : 5/592 (17 neurons)
Processing hyper voxel : 20/592 (16 neurons)
Processing hyper voxel : 545/592 (15 neurons)
Processing hyper voxel : 533/592 (14 neurons)
Processing hyper voxel : 565/592 (14 neurons)
Processing hyper voxel : 548/592 (13 neurons)
Processing hyper voxel : 17/592 (13 neurons)
Processing hyper voxel : 22/592 (12 neurons)
Processing hyper voxel : 25/592 (12 neurons)
Processing hyper voxel : 553/592 (11 neurons)
Processing hyper voxel : 550/592 (10 neurons)
Processing hyper voxel : 37/592 (10 neurons)
Processing hyper voxel : 9/592 (9 neurons)
Processing hyper voxel : 24/592 (9 neurons)
Processing hyper voxel : 561/592 (9 neurons)
Processing hyper voxel : 546/592 (8 neurons)
Processing hyper voxel : 4/592 (8 neurons)
Processing hyper voxel : 544/592 (8 neurons)
Processing hyper voxel : 8/592 (7 neurons

After ```SnuddaDetect``` has done the touch detection in the hyper voxels we use ```SnuddaProject``` to add the projections defined.

In [6]:
from snudda import SnuddaProject

sp = SnuddaProject(network_path=network_path)
sp.project()
sp.write()

pre dSPN, post iSPN


In [7]:
from snudda import SnuddaPrune

sp = SnuddaPrune(network_path=network_path)
sp.prune()


Synapses before pruning: 471
Synapses after pruning: 420 (89.17 % kept)
No gap junctions to prune.
